<img width="8%" alt="Google Sheets.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/Google%20Sheets.png" style="border-radius: 15%">

# Google Sheets - Update BDD Sales

**Tags:** #googlesheets #gsheet #data #naas_drivers #sales-engine #automation

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

**Description:** This notebook updates "Sales" database.

## Input

### Import libraries

In [1]:
from naas_drivers import gsheet
import pandas as pd
import os
from datetime import date
import naas_data_product
try:
    import openai
except:
    !pip install openai --user
    import openai
import time

✅ utils file '/home/ftp/abi/utils/data.ipynb' successfully loaded.
✅ utils file '/home/ftp/abi/utils/naas_chat_plugin.ipynb' successfully loaded.
✅ utils file '/home/ftp/abi/utils/naas_lab.ipynb' successfully loaded.
✅ utils file '/home/ftp/abi/utils/openai.ipynb' successfully loaded.


### Setup variables
**Inputs**
- `input_dir`: Input directory to retrieve file from.
- `file_reactions`: Name of the file with reactions to be retrieved.
- `file_comments`: Name of the file with comments to be retrieved.

**Outputs**
- `spreadsheet_url`: Google Sheets spreadsheet URL.
- `sheet_name`: Google Sheets sheet name.
- `append`: If False, data will be canceled and replaced.

In [7]:
# Inputs
input_dir = os.path.join(naas_data_product.OUTPUTS_PATH, "sales-engine", date.today().isoformat())
file_name = "sales"
openai_api_key = naas.secret.get("OPENAI_API_KEY")
spreadsheet_url = naas.secret.get("ABI_SPREADSHEET") or "YOUR_GOOGLE_SPREADSHEET_URL"
sheet_name_input = "GROWTH"
ref_contact_name = "LEADS"
ref_company_name = "LEADS_COMPANIES"

# Outputs
sheet_name_output = "SALES"
append = False

## Model

### Get data "GROWTH" from Google Sheets spreadsheet

In [8]:
df_input = gsheet.connect(spreadsheet_url).get(sheet_name=sheet_name_input)
print("Rows:", len(df_input))
df_input.head(1)

Rows: 3928


,ENTITY,SCENARIO,PLATFORM,FULLNAME,COHORT,OCCUPATION,INTERACTION_SCORE,PROFILE_URL,INTERACTIONS,LAST_INTERACTION_DATE,LAST_CONTENT_URL_INTERACTION,LAST_CONTENT_TITLE_INTERACTION
0,Travis Oliphant,W48-2023,LinkedIn,Curtis Whiting,W41-2023,"VP, Product at ZSuite Technologies",6,https://www.linkedin.com/in/ACoAAADHZOcBgU988t...,Comment 'We've had several interns at zsuite t...,Mon. 27 Nov.,https://www.linkedin.com/feed/update/urn:li:ac...,We can only offer this for free download for a...


### Get data "REF_CONTACT" from Google Sheets spreadsheet

In [9]:
ref_contact = gsheet.connect(spreadsheet_url).get(sheet_name=ref_contact_name)
print("Ref. contact:", len(ref_contact))
ref_contact.head(1)

Ref. contact: 2865


,FIRSTNAME,LASTNAME,FULLNAME,OCCUPATION,COMPANY,ICP,INTERACTION_SCORE,LAST_CONTENT_URL_INTERACTION,LAST_INTERACTION_DATE,LAST_CONTENT_TITLE_INTERACTION,PROFILE_URL,PUBLIC_ID,NOTES,CRM_COMPANY,CRM_CONTACT
0,Jérémy,Ravenel,Jérémy Ravenel,"Building @naas.ai, universal data & AI platfor...",naas.ai,Developer,64,https://www.linkedin.com/feed/update/urn:li:ac...,Wed. 29 Nov.,Quansight is #hiring. Know anyone who might be...,https://www.linkedin.com/in/ACoAAAJHE7sB5OxuKH...,jeremyravenel,Comment 'Anant asthana 👌can you share your fir...,NAASAI Inc,True


### Get data "REF_COMPANY" from Google Sheets spreadsheet

In [10]:
ref_company = gsheet.connect(spreadsheet_url).get(sheet_name=ref_company_name)
print("Ref. company:", len(ref_company))
ref_company.head(1)

Ref. company: 529


,COMPANY,STAFF_RANGE,INDUSTRY,COUNTRY,DIRECT_INTERACTIONS,INDIRECT_INTERACTIONS,INTERACTION_SCORE,LINKEDIN_URL,COMPANY_ID,COMPANY_NAME,COMPANY_URL,STAFF_COUNT,FOLLOWER_COUNT,CITY,WEBSITE,TAGLINE,DESCRIPTION
0,naas.ai,2-10,Software Development,FR,0,76,76,https://www.linkedin.com/company/naas-ai,70506391,naas.ai,https://www.linkedin.com/company/naas-ai,22,2990,Paris,https://www.naas.ai/,⚡️ All-in-one open source data platform\n#lowc...,Naas is the first Jupyter based data-science p...


### Create Sales dataframe

In [11]:
df = df_input.copy()

# Groupby
to_group = [
    "ENTITY",
    "SCENARIO",
    "LAST_INTERACTION_DATE",
    "PROFILE_URL",
]
to_agg = {
    "INTERACTION_SCORE": "sum"
}
df = df.groupby(to_group, as_index=False).agg(to_agg)
df = df.sort_values(by=["SCENARIO"], ascending=[True]).reset_index(drop=True)

# Cumsum
df["INTERACTION_CUM"] = df.groupby("PROFILE_URL", as_index=False)["INTERACTION_SCORE"].cumsum()

# Merge with Ref
ref = ref_contact[["FULLNAME", "OCCUPATION", "PROFILE_URL", "ICP", "COMPANY", "CRM_CONTACT"]]
df = pd.merge(df, ref, how="left")
df = df[(df["COMPANY"] != "TBD")]

ref2 = ref_company[["COMPANY", "COMPANY_NAME", "INDUSTRY", "STAFF_COUNT", "STAFF_RANGE", "COUNTRY"]]
df = pd.merge(df, ref2, how="left", left_on="COMPANY", right_on="COMPANY").fillna("NA")

# Filter data
df = df[
    (df["INTERACTION_CUM"] >= 3) &
    (df["COMPANY_NAME"] != "NA") &
    (~df["ICP"].isin(["NotICP", "NA"])) & 
    (df["CRM_CONTACT"].astype(str) == "False")
].drop_duplicates("PROFILE_URL")

# Cleaning
to_order = [
    "ENTITY",
    "SCENARIO",
    "LAST_INTERACTION_DATE",
    "FULLNAME",
    "OCCUPATION",
    "ICP",
    "COMPANY_NAME",
    "INDUSTRY",
    "STAFF_COUNT",
    "STAFF_RANGE",
    "COUNTRY",
    "PROFILE_URL",
    "INTERACTION_CUM",
]
to_rename = {
    "LAST_INTERACTION_DATE": "DATE",
    "INTERACTION_CUM": "INTERACTION_SCORE"
}
df = df[to_order].rename(columns=to_rename)
df = df.sort_values(by=["SCENARIO", "INTERACTION_SCORE"], ascending=[False, False]).reset_index(drop=True)

# # Prepare messaging
if not "MESSAGING_OPTIONS" in df.columns:
    df["MESSAGING_OPTIONS"] = "NA"
    
prompt_messaging = "Based on the following contact data, create 3 messaging options to engage with the person on LinkedIn, be brief, concise, casual but professional"
messagings = pload(input_dir, "messagings")
if messagings is None:
    messagings = {}
# for x in df["PROFILE_URL"].unique():
#     if x not in messagings:
#         print("Messaging for:", x)
#         contact = ref_contact[ref_contact["PROFILE_URL"] == x].reset_index(drop=True)
#         messaging = create_chat_completion(openai_api_key, prompt_messaging, str(contact.to_dict()))
#         messagings[x] = messaging
#         pdump(input_dir, messagings, "messagings")
        
df["MESSAGING_OPTIONS"] = df["PROFILE_URL"].map(messagings).fillna("NA")

# Display result
print("Rows:", len(df))
df.head(15)

Rows: 362


,ENTITY,SCENARIO,DATE,FULLNAME,OCCUPATION,ICP,COMPANY_NAME,INDUSTRY,STAFF_COUNT,STAFF_RANGE,COUNTRY,PROFILE_URL,INTERACTION_SCORE,MESSAGING_OPTIONS
0,Travis Oliphant,W48-2023,Mon. 27 Nov.,Aman Misri,Data Scientist @United Airlines | Ex-Musigma |...,Developer,United Airlines,Airlines and Aviation,63328,10001>,US,https://www.linkedin.com/in/ACoAAB1A7sgBZniof-...,4,"1. Hi Aman, I noticed you're a Data Scientist ..."
1,Travis Oliphant,W48-2023,Wed. 29 Nov.,Pete Bachant,Director of Software Engineering at WindESCo,Executive,WindESCo,IT Services and IT Consulting,34,11-50,US,https://www.linkedin.com/in/ACoAAAJCSNQB2vdr8a...,3,"1. Hi Pete, I noticed you liked a post about Q..."
2,Travis Oliphant,W48-2023,Wed. 29 Nov.,Joshua Chen,Computational Applied Math. Currently working ...,Developer,STR,Hospitality,618,1001-5000,US,https://www.linkedin.com/in/ACoAAAGrY_0B7I3m1u...,3,"1. Hi Joshua, I noticed your comment on our re..."
3,Travis Oliphant,W48-2023,Mon. 27 Nov.,Yashwant B,AI Intern @ Continental Automotive India | MTe...,Developer,Continental,Motor Vehicle Manufacturing,72651,10001>,DE,https://www.linkedin.com/in/ACoAADl0i7UBaReo1y...,3,"1. Hi Yashwant, I noticed your interest in our..."
4,Travis Oliphant,W48-2023,Wed. 29 Nov.,Samantha Walkow,Building and Researching Data Visualization an...,Developer,The Friends of the U.S. Law Library of Congress,Philanthropic Fundraising Services,4,11-50,US,https://www.linkedin.com/in/ACoAABd2R44BbByDo_...,3,"1. Hi Samantha, I noticed you liked our recent..."
5,Travis Oliphant,W48-2023,Wed. 29 Nov.,Charlie Haley,Software Engineering Manager,Executive,Wells Fargo,Financial Services,200991,10001>,US,https://www.linkedin.com/in/ACoAAAvwSRgBVEkBS8...,3,"1. Hi Charlie, I noticed you're interested in ..."
6,Travis Oliphant,W48-2023,Mon. 27 Nov.,Nils Senvalds,"Executive, Data Scientist, AI Solutions Archit...",Executive,Rogatio.ai,Business Consulting and Services,2,2-10,US,https://www.linkedin.com/in/ACoAAABJr4EBdFG8_S...,3,"1. Hi Nils, I noticed your interest in our Qua..."
7,Travis Oliphant,W48-2023,Mon. 27 Nov.,Kai Striega,Software Developer at BHP | Maintainer SciPy,Developer,BHP,Mining,37596,10001>,AU,https://www.linkedin.com/in/ACoAABPQwIkB1tPMJl...,3,1. Email: \nSubject: Let's Discuss Innovation ...
8,Travis Oliphant,W47-2023,Wed. 22 Nov.,Arthur Kendrick,Security Engineer II with Amazon | CISSP,Developer,Amazon,Software Development,850979,10001>,US,https://www.linkedin.com/in/ACoAAAQ_K00BY1gFCq...,6,"1. Hi Arthur, I noticed your interest in Pytho..."
9,Travis Oliphant,W47-2023,Wed. 22 Nov.,Florin Matei,Senior Software Developer,Developer,CCC Intelligent Solutions,Software Development,2290,1001-5000,US,https://www.linkedin.com/in/ACoAAAetNGIBeg4RAd...,6,"1. ""Hi Florin, I noticed your interest in Arti..."


## Output

### Send data to Google Sheets spreadsheet

In [12]:
gsheet.connect(spreadsheet_url).send(sheet_name=sheet_name_output, data=df, append=append)

{'insertedRow': 362}